In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torch
import os
from PIL import Image
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F 

In [2]:
label_dir = '../labels.txt'
f = open(label_dir,"r")
# And for reading use
classes = f.read()
f.close()
classes_list = classes.split('\n')
count = [int(i) for i in classes_list[::2]]
max_count = max(count)
weights = 1.0*max_count/np.array(count)
class_weights = torch.FloatTensor(weights).cuda()

In [3]:
import torchvision.models as models
model = models.densenet169(pretrained=True)
model.classifier = nn.Sequential(nn.Linear(1664, 359))


/opt/conda/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [4]:
# Setup: initialize the hyperparameters/variables
# Setup: initialize the hyperparameters/variables
num_epochs = 10           # Number of full passes through the dataset
batch_size = 32         # Number of samples in each minibatch
learning_rate = 0.01  
seed = np.random.seed(0) # Seed the random number generator for reproducibility
p_val = 0.1              # Percent of the overall dataset to reserve for validation
p_test = 0.2             # Percent of the overall dataset to reserve for testing


#TODO: Convert to Tensor - you can later add other transformations, such as Scaling here

transform = transforms.Compose([
        #transforms.RandomResizedCrop(224),
        transforms.ToPILImage('L'),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.RandomRotation(60),
        transforms.Resize([224,224],interpolation=2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [ ]:

# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

model = model
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)    

CUDA is supported
Model on CUDA? True


In [ ]:
### need to modify
# Setup the training, validation, and testing dataloaders
import quickdraw_dataloader as qd
from quickdraw_dataloader import create_split_loaders
root_dir = "../data_subset/"
train_loader, val_loader, test_loader = create_split_loaders(root_dir,batch_size, seed, transform=transform, 
                                                             p_val=p_val, p_test=p_test,
                                                             shuffle=True, show_sample=False, 
                                                             extras=extras)


In [ ]:
criterion = nn.CrossEntropyLoss(weight=class_weights).to(computing_device)

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
def validate(val_loader,model,optimizer,criterion):
    start = time.time()
    sum_loss = 0.0
    list_sum_loss = []
    correct_overall = 0.0
    minbatch_size = 0.0
    num = 0
    for mb_count, (val_images, val_labels) in enumerate(val_loader, 0):
        model.eval()
        with torch.no_grad():  
            optimizer.zero_grad()      
            val_images = torch.squeeze(torch.stack([val_images,val_images,val_images], dim=1, out=None))
            val_images, val_labels = val_images.to(computing_device), val_labels.to(computing_device)
            val_labels = val_labels.type(torch.cuda.FloatTensor)
            val_labels = val_labels.long()
            outputs = model(val_images)
            
            loss = criterion(outputs,val_labels)
            sum_loss += loss
            
            output_np = outputs.cpu().detach().numpy()
            label_np = val_labels.cpu().detach().numpy()
            correct_prediction, minbatch_size = correctness(label_np, output_np)
            correct_overall += correct_prediction
            minbatch_size += minbatch_size
    accuracy_vali = correct_overall/minbatch_size
    print('validatation accuracy',accuracy_vali) 
    
    print("validation time = ", time.time()-start)    
    return 1.0*sum_loss/mb_count, accuracy_vali

In [ ]:
def correctness(labels, out):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs==labels), labels.size
def train_model(model, criterion, optimizer, scheduler, num_epochs=10,resume=False, direct=""):
    since = time.time()
    total_loss = []
    if resume == False:
        avg_minibatch_loss = []
        total_vali_loss = []
        avg_training_accuracy = []
        avg_validation_accuracy = []
        epc_save = 0 # MAYBE
    else: 
        print('Resume model: %s' % direct)
        check_point = torch.load(direct)
        model.load_state_dict(check_point['state_dict'])
        optimizer.load_state_dict(check_point['optimizer'])
        avg_minibatch_loss = list(np.load('avg_minibatch_loss.npy'))
        total_vali_loss = list(np.load('total_vali_loss.npy'))
        avg_training_accuracy = list(np.load('avg_training_accuracy.npy'))
        avg_validation_accuracy = list(np.load('avg_validation_accuracy.npy'))
        epc_save = check_point['epoch']
        
    tolerence = 3
    i = 0 
    for epoch in range(epc_save,num_epochs):
        N = 100
        M = 700
        N_minibatch_loss = 0.0    
        best_loss = 100
        early_stop = 0
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        correct_overall = 0.0; 
        minbatch_size_overall = 0.0; 
        # Each epoch has a training and validation phase
        for phase in ['train']:
            scheduler.step()
            # Iterate over data.
            minibatch_time = time.time()
            for minibatch_count, (inputs, labels) in enumerate(train_loader, 0):
                inputs = torch.squeeze(torch.stack([inputs,inputs,inputs], dim=1, out=None))
                inputs = inputs.to(computing_device)
                labels = labels.to(computing_device)
                labels = labels.long()
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    labels = labels.long()
                    loss = criterion(outputs,labels)
                    #loss = criterion(torch.max(outputs,1)[1],torch.max(labels, 1)[1])
                    N_minibatch_loss += loss
                    # backward + optimize only if in training phase
                    loss.backward()
                    optimizer.step()
                    output_np = outputs.cpu().detach().numpy()
                    label_np = labels.cpu().detach().numpy()

                    correct_prediction, minbatch_size = correctness(label_np, output_np)
                    correct_overall += correct_prediction
                    minbatch_size_overall += minbatch_size
                # statistics
                # training stats
                if minibatch_count % N == 0 and minibatch_count!=0:    

                    # Print the loss averaged over the last N mini-batches    
                    N_minibatch_loss /= N
                    print('Epoch %d, average minibatch %d loss: %.3f' %
                        (epoch + 1, minibatch_count, N_minibatch_loss))
                    accuracy_train = correct_overall/minbatch_size_overall
                    print('accuracy',accuracy_train)
                    
                    
                    # Add the averaged loss over N minibatches and reset the counter
                    avg_minibatch_loss.append(N_minibatch_loss)
                    avg_minibatch_loss_1 = np.array(avg_minibatch_loss)
                    np.save('avg_minibatch_loss', avg_minibatch_loss_1)
                    avg_training_accuracy.append(accuracy_train)
                    avg_training_accuracy_1 = np.array(avg_training_accuracy)
                    np.save('avg_training_accuracy', avg_training_accuracy)
                    
                    N_minibatch_loss = 0.0
                    correct_overall = 0.0
                    minbatch_size_overall = 0.0
                    #print('accuracy, precision, recall', accuracy, precision, recall)
                    save_checkpoint({'epoch': epoch + 1,
                                'state_dict': model.state_dict(),
                                'optimizer': optimizer.state_dict(),
                                },
                                filename='./checkpoint/'+'%d_model_epoch%d.pth' % (epoch , minibatch_count))
                    print(N, "minibatch_time" , time.time()-minibatch_time)
                    minibatch_time = time.time()
                #Validation
                if minibatch_count % M == 0 and minibatch_count!=0:
                   
                    v_loss, accuracy_vali = validate(val_loader,model,optimizer,criterion)
                    print(v_loss.item())
                    
                    total_vali_loss.append(v_loss.item())
                    avg_validation_accuracy.append(accuracy_vali.item())
                    
                    total_vali_loss_1 = np.array(total_vali_loss)
                    np.save('total_vali_loss', total_vali_loss_1)      
                    avg_validation_accuracy_1 = np.array(avg_validation_accuracy)
                    np.save('avg_validation_accuracy', avg_validation_accuracy_1)      
                
                    if total_vali_loss[i] > best_loss and i != 0:
                        early_stop += 1
                        if early_stop == tolerence:
                            print('early stop here')
                            break
                    else:
                        best_loss = total_vali_loss[i] 
                        early_stop = 0
                    i = i + 1
            print("Finished", epoch + 1, "epochs of training")
    print("Training complete after", epoch, "epochs")
    
    print("total_vali_loss")
    print(total_vali_loss)
    print("avg_minibatch_loss")
    print(avg_minibatch_loss)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s '.format(
        time_elapsed // 60, time_elapsed % 60))


In [ ]:
def save_checkpoint(state, is_best=0, filename='models/checkpoint.pth.tar'):
    torch.save(state, filename)


In [ ]:
model_trained = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=5,resume=False,direct='')


Epoch 0/4
----------
Epoch 1, average minibatch 1 loss: 11.953
accuracy 0.0
1 minibatch_time 2.0727810859680176
validatation accuracy 11.25
validation time =  393.77547955513
tensor(5.9683, device='cuda:0')
Epoch 1, average minibatch 2 loss: 5.937
accuracy 0.03125
1 minibatch_time 394.89313316345215
validatation accuracy 17.291666666666668
validation time =  394.7303397655487
tensor(5.9346, device='cuda:0')
Epoch 1, average minibatch 3 loss: 5.985
accuracy 0.03125
1 minibatch_time 395.86189675331116


In [ ]:
# import matplotlib.pyplot as plt
# from IPython.display import display # to display images
# for minibatch_count, (inputs, labels) in enumerate(train_loader, 0):
#     print(inputs.shape)
#     #inputs = inputs.to(computing_device)
#     #labels = labels.to(computing_device)
#     image = inputs.cpu().detach().numpy()[1].reshape((224,224))*255
    
#     print(image.sum())
#     plt.imshow(image.astype(int),)
#     plt.show()
#     image = inputs.cpu().detach().numpy()[0].reshape((224,224))*255
    
#     print(image.sum())
#     plt.imshow(image.astype(int),)
#     plt.show()
#     image = inputs.cpu().detach().numpy()[2].reshape((224,224))*255
    
#     print(image.sum())
#     plt.imshow(image.astype(int),)
#     plt.show()
    
#     image = inputs.cpu().detach().numpy()[3].reshape((224,224))*255
    
#     print(image.sum())
#     plt.imshow(image.astype(int),)
#     plt.show()
#     #image = Image.fromarray(image.astype(int),"L")
#     #display(image)
#     if minibatch_count >10: 
#         break
    

In [ ]:
# >>> loss = nn.CrossEntropyLoss()
# >>> input = torch.randn(3, 5, requires_grad=True)
# >>> target = torch.empty(3, dtype=torch.long).random_(5)
# >>> output = loss(input, target)
# >>> output.backward()

In [ ]:
# input